In [45]:
import cantera as ct
import numpy as np
import scipy as sp
import pandas as pd
from pint import UnitRegistry
import plotly.express as px
import plotly.graph_objects as go
# import pypropep as ppp
import yaml
import time
import CEA_Wrap as CEA
import copy

ureg = UnitRegistry()
Q = ureg.Quantity

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 12)


In [32]:
class EngineState:
    '''
    This class will define the properties of an rocket engine at the chamber, throat and exit 
    given the propellant, and conditions that engine is in.

    Parameters
    --------------------------------------------------
    oxidizer : string
        A string of a species name which is used to define a cantera solution representing the oxidizer.
        Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
    fuel : string
        Similar to oxidizer, this is a string of a species name which is used to define a cantera solution representing the fuel.
        Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
    of_ratio : float
        the mass ratio of oxidizer to fuel defined as mass_oxidizer / mass_fuel.
    pressure : float
        The pressure of the combustion chamber in units of Pascals (Pa).
    exit_value : float
        The exit conditions of the engine. Depending on what 'exit parameter' is
        defined to be the exit condition can be defined in the following ways:
            'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
            'area ratio': Defined as area_exit / area_throat should be value greater than 1 TODO: add functionality
    exit_parameter : string
        A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
        or 'area ratio'. 
    temp_oxidizer : float
        A positive float value defining the temperature of the oxidizer in Kelvin (K). If the oxidizer is liquid the temperature 
        is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
    temp_fuel : float
        Similar to above. positive float value defining the temperature of the fuel in Kelvin (K). If the fuel is liquid the temperature 
        is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
    transport_species Optional: [list | None]
        A list of species 
    assumption : str
        The assumption determines the state of the combustion products and whether their composition is in equilibrium, or fixed/frozen:
            'equilibrium' (default): Composition is always in equilibrium
            'frozen': Composition is fixed after initial combustion in the chamber. 

        

    TODO board: 
    --------------------------------------------------
        TODO: apply efficiency.
        TODO: apply flow seperation
        TODO: improve chemical database
        TODO: allow fluid temp setting
        TODO: condensed species capabilities
        TODO: Improve convergence *especially for locations along the length of the chamber*
        TODO: Ambient pressure
        TODO: Ambient pressure Array
        TODO: add Pint https://pint.readthedocs.io/en/stable/advanced/wrapping.html#wrapping
        TODO: add FAC capabilities
        TODO: add frozen

    Attributes:
    --------------------------------------------------
    Properties : Pandas Dataframe
    
    Methods
    --------------------------------------------------

    A class for the state of a rocket engine. Given the propellants and conditions of a rocket engine this 
    class will define the properties of an engine at
    '''

    def __init__(self, oxidizer, fuel, of_ratio, pressure, exit_value, gas, condensate = None, size_value=None, assumption = 'equilibrium', size_parameter ="thrust",  
                 exit_parameter="pressure", transport=None, throat_inlet_radius_ratio=1.5, temperature_wall=None):
        '''
        Initializes instance of class. See class description for details. 
        '''
        # initializes propellants
        self.assumption = assumption
        self._of_ratio = of_ratio
        self._pressure = pressure
        start_time = time.time()
        oxidizer.TP = oxidizer.T, pressure
        oxidizer.equilibrate('TP')
        self._oxidizer = self.__clone_solution(oxidizer)
        fuel.TP = fuel.T, pressure
        fuel.equilibrate('TP')
        self._fuel = self.__clone_solution(fuel)
        self.chamber_gas = self.__clone_solution(gas)
        self.throat_gas = self.__clone_solution(gas)
        self.exit_gas = self.__clone_solution(gas)
        self._gas = self.__clone_solution(gas)

        if condensate:
            self._condensate = self.__clone_solution(condensate)
            self.exit_condensate = self.__clone_solution(condensate)
            self.throat_condensate = self.__clone_solution(condensate)
            self.chamber_condensate = self.__clone_solution(condensate)
        else:
            self._condensate = None
            self.exit_condensate = None
            self.throat_condensate = None
            self.chamber_condensate = None
        if transport: 
            self._transport = self.__clone_solution(transport)
            self.exit_transport = self.__clone_solution(transport)
            self.throat_transport = self.__clone_solution(transport)
            self.chamber_transport = self.__clone_solution(transport)
        else: 
            self._transport = None
            self.exit_transport = None
            self.throat_transport = None
            self.chamber_transport = None

    
        # finds chamber, throat, and exit properties
        chem_init_time = time.time()- start_time
        self.__chamber_properties(self.chamber_gas, self.chamber_condensate, self.chamber_transport)
        chamber_time = time.time()- start_time - chem_init_time
        self.__throat_properties(self.throat_gas, self.throat_condensate, self.throat_transport)
        throat_time = time.time()- start_time - chamber_time -chem_init_time
        self.__exit_properties(self.throat_gas, self.throat_condensate, self.throat_transport, exit_value, exit_parameter=exit_parameter)
        exit_time = time.time()- start_time - throat_time - chamber_time -chem_init_time

        self.throat_inlet_radius_ratio = throat_inlet_radius_ratio # this value is not needed for sizing but is necessary to find heat tranfer coeff. via bartz
        self.temperature_wall = temperature_wall
        
        # sizes engine
        if  size_value:
            self.size_engine(size_value, size_parameter=size_parameter)
        size_time = time.time()- start_time - exit_time - throat_time - chamber_time -chem_init_time


        # self.engine_state_dict = {'chamber' : self.chamber, 'throat' : self.throat, 'exit' : self.exit }
        self.engine_state = pd.DataFrame([self.chamber, self.throat, self.exit], index=["chamber", "throat", "exit"])

        df_time = time.time()- start_time-size_time - exit_time - throat_time - chamber_time - chem_init_time

        print(f"-----\ntotal time: {time.time()-start_time}\nchemistry initialization: {chem_init_time}\n chamber properties: {chamber_time}\nthroat time: {throat_time}\nexit time: {exit_time}\nsizing time: {size_time}\ndf making: {df_time} ")

    def __call__(self):
        return self.engine_state
    
    def __str__(self):
        return self.engine_state.to_string()
    
    def __clone_solution(self, sol: ct.Solution) -> ct.Solution:
        """Return a deep, independant copy of a Cantera Solution"""
        new = ct.Solution(thermo=sol.thermo_model, species=sol.species())
        if sol.transport_model != 'none': 
            new.transport_model = sol.transport_model
        new.TPX = sol.T, sol.P, sol.X
        return new

    def __get_thermo_derivatives(self, gas, condensate):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species this method will find thermodynamic derivatives.
        These derivatives can then be used to calculate thermodynamic properties, or create interpolation/extrapolation of engine properties 

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.
        The equations used are 2.50, 2.51, 2.56, 2.57, 2.58, 2.64, 2.65, 2.66 

        TODO: add condensed species to implementation
        TODO: explore interpolation of engine properties using derivatives
        
        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        
        Returns
        --------------------------------------------------
        Derivatives : dictionary:
            A dictionary consisting of the derivatives this function calculates. The keys and descritions of said dictionaries are as follows
        
            dpi_dlnT_P : list
                A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
                and lambda is the langrangian multiplier
            dlnn_dlnT_P : float
                The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas
            dpi_dlnP_T : list
                A list of the derivative of pi with respect to ln(P) at constant temperature for each element.
            dlnn_dlnP_T : float
                The derivative of ln(n) with respect to ln(P) at constant tempearture.
            dlnV_dlnT_P : float
                The derivative of ln(V) with respect to ln(T) at constant pressure.
            dlnV_dlnP_T : float
                The derivative of ln(V) with respect to ln(P) at constant tempearture.
        '''

        if self.assumption == 'equilibrium':             
            # Defines the number of moles of each species in the mixture
            moles = gas.X * (1/ gas.mean_molecular_weight)
            num_variables = 2 * gas.n_elements + 2

            # Initializing Solution Matrices table 2.3 and 2.4 in RP-1311
            coeff_matrix = np.zeros((num_variables, num_variables))
            right_hand_side = np.zeros(num_variables)

            # Initializes a_ij
            a_ij = np.zeros((gas.n_elements, gas.n_species))
            for i, element in enumerate(gas.element_names): 
                for j, species in enumerate(gas.species_names):
                    a_ij[i,j] = gas.n_atoms(species, element)


            # Coefficients for equation 2.56 TODO: add terms for condensed species
            for k in range(gas.n_elements):
                for i in range(gas.n_elements):
                    coeff_matrix[k,i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
                coeff_matrix[k, gas.n_elements] = np.sum(a_ij[k,:] * moles)
                right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)

            # TODO add equation 2.57 (it is for condensed species)
            # for i in range():
            #     None


            # Coefficients for equation 2.58 
            for i in range(gas.n_elements):
                coeff_matrix[gas.n_elements, i] = np.sum(a_ij[i, :] * moles)
            right_hand_side[gas.n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

            # Coefficients for equation 2.64 TODO: add terms for condensed species
            for k in range(gas.n_elements):
                for i in range(gas.n_elements):
                    coeff_matrix[gas.n_elements+1+k,gas.n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
                coeff_matrix[gas.n_elements+1+k, 2*gas.n_elements+1] = np.sum(a_ij[k,:] * moles)
                right_hand_side[gas.n_elements+1+k] = np.sum(a_ij[k,:] * moles)

            # TODO add equation 2.65 (it is for condensed species)
            

            # Coefficeints for equation 2.66
            for i in range(gas.n_elements):
                coeff_matrix[2*gas.n_elements+1, gas.n_elements+1+i] = np.sum(a_ij[i, :] * moles)
            right_hand_side[2*gas.n_elements+1] = np.sum(moles)

            # Solve for the derivatives define them based off table 2.3, 2.4 and equation 2.50 and 2.51 
            derivs = np.linalg.solve(coeff_matrix, right_hand_side)
            derivatives = { "dpi_dlnT_P"    : derivs[0 : gas.n_elements], 
                            "dlnn_dlnT_P"   : derivs[gas.n_elements], 
                            "dpi_dlnP_T"    : derivs[gas.n_elements + 1: 2 * gas.n_elements + 1],
                            "dlnn_dlnP_T"   : derivs[2 * gas.n_elements + 1], 
                            "dlnV_dlnT_P"   : 1 + derivs[gas.n_elements], 
                            "dlnV_dlnP_T"   : -1 + derivs[2 * gas.n_elements + 1]}

        elif self.assumption == 'frozen': 
            derivatives = { "dpi_dlnT_P"    : [np.nan]*gas.n_elements, 
                            "dlnn_dlnT_P"   : 0,
                            "dpi_dlnP_T"    : [np.nan]*gas.n_elements,
                            "dlnn_dlnP_T"   : 0, 
                            "dlnV_dlnT_P"   : 1, 
                            "dlnV_dlnP_T"   : -1}
        else: 
            raise ValueError("Assumption must be 'frozen' or 'equilibrium'")

        return derivatives

    def __get_thermo_properties(self, gas, condensate, dpi_dlnT_P, dlnn_dlnT_P, dlnV_dlnT_P, dlnV_dlnP_T):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic derivatives of said mixture, this function will find 
        the thermodynamic properties of the mixture

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        TODO: add condensed species to implementation
        TODO: add capability to solve for thermal conductivity, viscocity, and prandtl number  

        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        dpi_dlnT_P : list
            A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
            and lambda is the langrangian multiplier
        dlnn_dlnT_P : float
            The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas.
        dlnV_dlnT_P : float
            The derivative of ln(V) with respect to ln(T) at constant pressure.
        dlnV_dlnP_T : float
            The derivative of ln(V) with respect to ln(P) at constant tempearture.
            
        Returns
        --------------------------------------------------
        properties : dictionaries
            A dictionary consisting of the properties this function calculates. The keys and descritions of said dictionaries are as follows

            Pressure : float
            Temperature : float
            density : float
            specific_volume : float
            enthalpy : float
            internal_energy : float
            gibbs : float
            entropy : float
            molar_mass : float
            c_p : float
                the specific heat at constant pressure
            c_v : float
                the specific heat at constant volume
            gamma : float
                the specific heat ratio
            gamma_s : float
                defined as derivative ln(P) with respect to ln(rho) at constant entropy per equation 2.71 in RP-1311
            speed_sound : float
                the speed of sound in the mixture
        '''

        if self.assumption == 'equilibrium':
            # Defines the number of moles of each species in the micture
            moles = gas.X * (1/ gas.mean_molecular_weight)

            # Initializes a_ij
            a_ij = np.zeros((gas.n_elements, gas.n_species))
            for i, element in enumerate(gas.element_names): 
                for j, species in enumerate(gas.species_names):
                    a_ij[i,j] = gas.n_atoms(species, element)

            # Finds specific heat at constant pressure based on equation 2.59 TODO: add terms for condensed species
            c_p =  ct.gas_constant * (
                np.sum([dpi_dlnT_P[i] * np.sum(a_ij[i,:] * moles * gas.standard_enthalpies_RT) for i in range(gas.n_elements)]) +
                np.sum(moles * gas.standard_enthalpies_RT) * dlnn_dlnT_P +
                np.sum(moles * gas.standard_cp_R) +
                np.sum(moles * gas.standard_enthalpies_RT**2)
            )

            # Finds specifc heat at constant volume, based on equation 2.70, specific heat ratio, The isentropic exponent based on equation 2.73, 
            # and speed of sound based on equation 2.74
            c_v = c_p + gas.P * gas.v / gas.T * dlnV_dlnT_P**2 / dlnV_dlnP_T
            gamma = c_p / c_v
            gamma_s = -gamma/dlnV_dlnP_T
            speed_sound = np.sqrt(ct.gas_constant * gas.T * gamma_s/gas.mean_molecular_weight)
        
        elif self.assumption == 'frozen':
            c_p = gas.cp
            c_v = gas.cv
            gamma = c_p / c_v
            gamma_s = gamma
            speed_sound = np.sqrt(ct.gas_constant * gas.T * gamma_s/gas.mean_molecular_weight)
        
        else: 
            raise ValueError("Assumption must be 'frozen' or 'equilibrium'")

        properties = {  "of ratio"          : self._of_ratio,
                        "pressure"          : gas.P,
                        "temperature"       : gas.T,
                        "density"           : gas.density_mass,
                        "specific volume"   : gas.volume_mass,
                        "enthalpy"          : gas.enthalpy_mass,
                        "internal energy"   : gas.int_energy_mass,
                        "gibbs"             : gas.gibbs_mass,
                        "entropy"           : gas.entropy_mass,
                        "molar mass"        : gas.mean_molecular_weight,
                        "c_p"               : c_p,
                        "c_v"               : c_v,
                        "gamma"             : gamma,                                                                            
                        "gamma_s"           : gamma_s,    
                        "speed sound"       : speed_sound          
                        }

        return properties
    
    def __get_transport_properties(self, gas, transport):
        '''
        TODO: Update  comment
        TODO: find equilibrium thermal conductivity
        '''
        if transport:
            transport.TPX = gas.T, gas.P, {k: v for k, v in zip(gas.species_names, gas.X) if k in transport.species_names}        
            # frozen transport properties
            properties = {  "viscosity"             : transport.viscosity,                                                  # frozen viscosity in Pa*s
                            "thermal conductivity"  : transport.thermal_conductivity,                                       # thermal conductivity in W/m/K
                            "prandtl number"        : transport.cp * transport.viscosity / transport.thermal_conductivity   # frozen prandtl number
                            }
        else: 
            properties = {  "viscosity"             : np.nan,
                            "thermal conductivity"  : np.nan,
                            "prandtl number"        : np.nan
                            }
        # TODO: implement reacting transport properties
        

        return properties

    def _heat_flux(self, temperature, pressure, transport_mole_fractions, area, mach, viscosity_exponent = 0.6):

        if not self.temperature_wall:
            print(  "temperature_wall is not defined. If you want heat transfer coeff. try initiating class with temperature wall defined, or " \
                    "setting [object_name].temperature_wall to chosen value.")
            heat_prop = {'heat transfer coefficient': np.nan,
                         'heat flux': np.nan} 
        else: 
            T_ref = temperature * (1+0.032* mach**2 + 0.58*(self.temperature_wall/temperature - 1))
            self._transport.TPX = T_ref, pressure, transport_mole_fractions

            prandtl_number = self._transport.viscosity * self._transport.cp / self._transport.thermal_conductivity
            d_t = self.throat['diameter']
            r_t_inlet = self.throat_inlet_radius_ratio*d_t/2
            m = viscosity_exponent #exponent of temperature such that mu proportional to T^m

            sigma = 1/ ( 1/2*self.temperature_wall/self.chamber['temperature'] * (1+ (self.chamber['gamma']-1)/2 * mach**2)+ 1/2)**(0.8-m/5) / (1+(self.chamber['gamma']-1)/2 * mach**2)**(m/5)
            
            heat_transfer_coeff =   (0.026/d_t**0.2 * (self._transport.viscosity**0.2*self.chamber['c_p']/prandtl_number**0.6) 
                                    * (self.chamber['pressure']/self.chamber['c*'])**0.8* (d_t/r_t_inlet)**0.1 * ((np.pi*d_t**2/4)/area)**0.9* sigma)

            T_aw = self.chamber['temperature'] * (1+ prandtl_number**0.33*(self.chamber['gamma']-1)/2*mach**2)/(1+(self.chamber['gamma']-1)/2*mach**2)

            heat_prop = {'heat transfer coefficient': heat_transfer_coeff,
                         'heat flux': heat_transfer_coeff * (T_aw-self.temperature_wall)} 

        return heat_prop

    def __chamber_properties(self, gas, condensate, transport):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic properties of said mixture, this function will find 
        properties of the engine

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        Parameters
        --------------------------------------------------
        products : Cantera Mixture 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
            
        Returns
        --------------------------------------------------
        properties : dictionary
        '''

        # Equilibriates chamber returning a cantera mixture with the properties & composition at the chamber
        molar_ratio = self._of_ratio / (self._oxidizer.mean_molecular_weight / self._fuel.mean_molecular_weight)
        moles_ox = molar_ratio / (1 + molar_ratio)
        moles_f = 1 - moles_ox

        chamber_mixture = ct.Mixture([(self._fuel, moles_f), (self._oxidizer, moles_ox), (gas, 0)])
        chamber_mixture.equilibrate('HP', estimate_equil=-1)
        # Finds thermodynamic derivatives 
        derivatives = self.__get_thermo_derivatives(gas, condensate)

        # Finds thermodynamic properties
        therm_prop = self.__get_thermo_properties(gas, condensate, derivatives["dpi_dlnT_P"], derivatives["dlnn_dlnT_P"], derivatives["dlnV_dlnT_P"], derivatives["dlnV_dlnP_T"])
        
        # Finds tranpsort properties
        transport_prop = self.__get_transport_properties(gas, transport)

        # Calculate c* per
        char_velocity = (np.sqrt(ct.gas_constant * therm_prop["temperature"] / (therm_prop["molar mass"] * therm_prop["gamma"])) * 
                            np.power(2 / (therm_prop["gamma"] + 1), -(therm_prop["gamma"] + 1) / (2*(therm_prop["gamma"] - 1))))

        # velocity and Mach are 0 in a FAC combustor, area ratio, Isp, Ivac, and Cf are not defined at chamber
        chamber_prop = {"velocity"        : 0, 
                        "mach"            : 0, 
                        "area ratio"      : np.nan,
                        "I_sp"            : np.nan,
                        "I_vac"           : np.nan,
                        "c*"              : char_velocity,
                        "C_f"             : np.nan,
                        "mole fraction"   : {k:v for k,v in gas.mole_fraction_dict().items() if v > 10e-6}}
        
        self.chamber = therm_prop | transport_prop | chamber_prop | derivatives

    def __throat_properties(self, gas, condensate, transport):
        '''
        Description
        --------------------------------------------------
        This is an internal method not meant for use outside the class.
        given a solution of gases & condensed species, this function will find 
        properties of the engine

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        Parameters
        --------------------------------------------------
        products : Cantera Solution 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
            
        Returns
        --------------------------------------------------
        None
        '''
        chamber = self.chamber
        # initial guess at throat pressure using specific heat ratio gamma
        pressure_throat = chamber["pressure"] / np.power((chamber["gamma_s"] + 1) / 2., chamber["gamma_s"] / (chamber["gamma_s"] - 1))

        # Setting up for iteration
        max_iter_throat = 10            # NOTE exceeds value of 4 from RP-1311
        tolerance_throat = 4e-5
        mach = 1.0
        num_iter = 0
        residual = 1

        gas.SPX = chamber["entropy"], pressure_throat, self.chamber_gas.X
 
        while (residual > tolerance_throat and num_iter < max_iter_throat ) :
            num_iter += 1
            gas.SP = chamber["entropy"], pressure_throat

            if self.assumption == 'equilibrium': 
                gas.equilibrate('SP')   
            elif self.assumption == 'frozen':
                pass
            else:
                raise ValueError("Assumption must be 'frozen' or 'equilibrium'")
            
            throat_derivatives = self.__get_thermo_derivatives(gas, condensate)
            throat_properties = self.__get_thermo_properties(gas, condensate, throat_derivatives["dpi_dlnT_P"], throat_derivatives["dlnn_dlnT_P"], throat_derivatives["dlnV_dlnT_P"], throat_derivatives["dlnV_dlnP_T"])
            
            velocity = np.sqrt(2 * (chamber["enthalpy"] - throat_properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * throat_properties["temperature"] * throat_properties["gamma_s"]  / throat_properties["molar mass"])
            mach = velocity / speed_sound
            
            pressure_throat = pressure_throat * (1 + throat_properties["gamma_s"] * mach**2) / (1 + throat_properties["gamma_s"] )

            residual = np.abs((velocity**2 - speed_sound**2)/velocity**2)

        if num_iter >= max_iter_throat:
            print(f'Warning: Convergance took {num_iter} iterations which exceeds the limit of {max_iter_throat} max iterations. residual is {residual} which exceeds tolerance of {tolerance_throat}.')
        
        transport_prop = self.__get_transport_properties(gas, transport)
        
        # velocity and Mach are the same at throat, area ratio, Isp, Ivac, and Cf are not defined at chamber
        throat_prop = { "velocity"        : speed_sound, 
                        "mach"            : 1, 
                        "area ratio"      : np.nan,
                        "I_sp"            : np.nan,
                        "I_vac"           : np.nan,
                        "c*"              : np.nan,
                        "C_f"             : np.nan,
                        "mole fraction"   : {k:v for k,v in gas.mole_fraction_dict().items() if v > 10e-6}}

        self.throat = throat_properties | transport_prop | throat_prop | throat_derivatives

    def __exit_properties(self, gas, condensate, transport, exit_value, exit_parameter='pressure'):
        '''
        This is an internal method not meant for use outside the class.
        equilbriates solution and finds thermal derivatives and properties at the exit of the engine.
      
        Parameters
        --------------------------------------------------
        products : Cantera Solution 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        exit_value : float
            The exit conditions of the engine. Depending on what 'exit parameter' is
            defined to be the exit condition can be defined in the following ways:
                'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
                'area ratio': Definfed as area_exit / area_throat should be value greater than 1 TODO: add functionality
        exit_parameter : string
            A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
            or 'area ratio'. 
            
        Returns
        --------------------------------------------------
        None
        '''

        if exit_parameter == "area ratio":
            exit_properties = self.state_at_area(gas, condensate, transport, exit_value, speed = "supersonic")
            ae_mdot = 1/(exit_properties["density"]*exit_properties["velocity"])

            exit_properties["I_sp"] = exit_properties["velocity"] / sp.constants.g
            exit_properties["I_vac"] = exit_properties["velocity"] / sp.constants.g + exit_properties["pressure"] * ae_mdot / sp.constants.g
            exit_properties["C_f"] = exit_properties["velocity"] / self.chamber["c*"]

            self.exit =  exit_properties
        
        elif exit_parameter == "pressure":
            pressure = exit_value
            gas.SPX = self.chamber["entropy"], pressure, self.throat_gas.X
            if self.assumption == 'equilibrium': 
                gas.equilibrate('SP')   
            elif self.assumption == 'frozen':
                pass
            else:
                raise ValueError("Assumption must be 'frozen' or 'equilibrium'")

            exit_derivatives = self.__get_thermo_derivatives(gas, condensate)
            exit_properties = self.__get_thermo_properties(gas, condensate, exit_derivatives["dpi_dlnT_P"], exit_derivatives["dlnn_dlnT_P"], exit_derivatives["dlnV_dlnT_P"], exit_derivatives["dlnV_dlnP_T"])

            velocity = np.sqrt(2* (self.chamber["enthalpy"] - exit_properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * exit_properties["temperature"] * exit_properties["gamma_s"]  / exit_properties["molar mass"])

            at_mdot = 1 / (self.throat["density"]*self.throat["velocity"])
            ae_mdot = 1 / (exit_properties["density"]*velocity)
            ae_at = ae_mdot/at_mdot
            Isp = velocity / sp.constants.g
            Ivac = Isp + pressure * ae_mdot / sp.constants.g
            Cf = velocity / self.chamber["c*"]

            transport_prop = self.__get_transport_properties(gas, transport)

            exit_prop = {   "velocity"        : velocity, 
                            "mach"            : velocity/speed_sound, 
                            "area ratio"      : ae_at,
                            "I_sp"            : Isp,
                            "I_vac"           : Ivac,
                            "c*"              : np.nan,
                            "C_f"             : Cf,
                            "mole fraction"   : {k:v for k,v in gas.mole_fraction_dict().items() if v > 10e-6}}

            exit_properties = exit_properties | transport_prop | exit_prop | exit_derivatives

            self.exit = exit_properties
            
        else: 
            raise ValueError("Invalid input. exit_parameter was not defined as 'pressure' or 'area ratio'")

    def size_engine(self, size_value, size_parameter = 'thrust'): 
        '''
        Description
        --------------------------------------------------
        Given thrust, throat diameter, or massflow will define the other three and update engine_state accordingly

        Parameters
        --------------------------------------------------
        size_value : float
            Depending on what 'size_parameter' is defined in the following ways
                'thrust' : the target thrust of the engine in units newtons
                'mass flow' : the target mass flow of the engine in units kilogram/second
                'throat diameter' : the diameter of the throat in units meter
        size_parameter : string
            a string with three possible values:
                'thrust' 
                'mass flow'
                'throat diameter' 

        Returns
        --------------------------------------------------
        None
        '''

        if size_parameter == 'thrust': 
            thrust = size_value
            mass_flow = thrust / self.exit['velocity']
            throat_area = mass_flow / (self.throat['velocity'] * self.throat['density'])
            # throat_area_2 = eneter ideal compressible choked equation
            # throat_area_3 = thrust / (self.exit['C_f'] * self.chamber['pressure'])
            throat_diameter = np.sqrt(throat_area/np.pi) * 2
            # throat_diameter_2 = np.sqrt(throat_area_2/np.pi) * 2
            # throat_diameter_3 = np.sqrt(throat_area_3/np.pi) * 2


        elif size_parameter == 'mass flow':
            mass_flow = size_value
            thrust = mass_flow * self.exit['velocity']
            throat_area = mass_flow / (self.throat['velocity'] * self.throat['density'])
            throat_diameter = np.sqrt(throat_area/np.pi) * 2
        elif size_parameter == 'throat diameter':
            throat_diameter = size_value
            throat_area = np.pi * (throat_diameter/2)**2
            mass_flow = throat_area * self.throat['velocity'] * self.throat['density']
            thrust = mass_flow * self.exit['velocity']
        else:
            raise ValueError("Invalid input. size_parameter was not defined as 'thrust', 'mass flow' or 'throat diameter'")

        chamber_size = {'thrust' :      np.nan,
                        'mass flow' :   mass_flow,
                        'area' :        np.inf,
                        'diameter' :    np.inf}
        
        throat_size =  {'thrust' :      np.nan,
                        'mass flow' :   mass_flow,
                        'area' :        throat_area,
                        # 'area 2' :      throat_area_2,
                        # 'area 3' :      throat_area_3,
                        'diameter' :    throat_diameter}
                        # 'diameter 2' :  throat_diameter_2,
                        # 'diameter 3' :  throat_diameter_3

        exit_size =    {'thrust' :      thrust,
                        'mass flow' :   mass_flow,
                        'area' :        throat_area * self.exit['area ratio'],
                        'diameter' :    np.sqrt(throat_area * self.exit['area ratio']/np.pi) * 2}
        
        self.chamber.update(chamber_size)
        self.throat.update(throat_size)
        self.exit.update(exit_size)

        self.chamber.update({'heat transfer coefficient': np.nan,'heat flux': np.nan})
        self.throat.update(self._heat_flux(self.throat["temperature"],self.throat["pressure"], self.throat_transport.X, self.throat['area'], self.throat['mach']))
        self.exit.update(self._heat_flux(self.exit["temperature"],self.exit["pressure"], self.exit_transport.X, self.exit['area'], self.exit['mach']))

        self.engine_state = pd.DataFrame([self.chamber, self.throat, self.exit], index=["chamber", "throat", "exit"])
            
    def state_at_area(self, gas, condensate, transport, area_ratio, speed = "supersonic"):
        '''
        Description
        --------------------------------------------------
        Iteratively find the properties at a given location in along the nozzle given the area ratio and whether it is in the subsonic or supersonic section of the nozzle.

        Parameters
        --------------------------------------------------
        area_ratio : float
            The ratio at of the area at a given location over the area of the throat.
        speed : string
            a string with two possible values:
                'subsonic' : the converging section of the nozzle
                'supersonic' : the diverging section of the nozzle        

        Returns
        --------------------------------------------------
        local_properties : Dict
            A dictionary of the thermodynamic properties and derivatives 
        
        '''

        # checking for valid input
        if area_ratio <= 1:
            raise ValueError("Area ratio was less than or equal to 1")
        
        # getting initial guess of ln(pc/pe).
        if speed == "subsonic": 
            if area_ratio > 1.000 and area_ratio < 1.09:
                lnpc_p = 0.9*np.log(self.chamber["pressure"]/self.throat["pressure"])/(area_ratio+10.587*np.log(area_ratio)**3+9.454*np.log(area_ratio))
            elif area_ratio >= 1.09:
                lnpc_p = np.log(self.chamber["pressure"]/self.throat["pressure"])/(area_ratio+10.587*np.log(area_ratio)**3+9.454*np.log(area_ratio)) 
                
        if speed == "supersonic": 
            if area_ratio > 1.000 and area_ratio < 2:
                lnpc_p = np.log(self.chamber["pressure"]/self.throat["pressure"]) +np.sqrt(3.294*np.log(area_ratio)**2+1.534*np.log(area_ratio))
            elif area_ratio >= 2:
                lnpc_p = self.chamber["gamma_s"] + 1.4 * np.log(area_ratio)
        
        # initial guess at equilibrium
        pressure = self.chamber["pressure"]/np.exp(lnpc_p)
        gas.SPX = self.throat["entropy"], pressure, self.throat["mole fraction"]
        if self.assumption == 'equilibrium': 
                gas.equilibrate('SP')   
        elif self.assumption == 'frozen':
            pass
        else:
            raise ValueError("Assumption must be 'frozen' or 'equilibrium'")            

        # defining iteration limits and convergence
        num_iter = 0
        max_iter = 10
        tolerance = 4e-5
        residual = 1

        # defines throat area / throat massflow
        at_mdot = 1 / (self.throat["density"]*self.throat["velocity"])

        # iterative solver for state at position _blank_
        while residual > tolerance:
            num_iter += 1

            if num_iter == max_iter:
                print(f"exceeded {max_iter} iterations, residual is {residual} which is not below tolerance of {tolerance}")
                break

            derivatives = self.__get_thermo_derivatives(gas, condensate)
            properties = self.__get_thermo_properties(gas, condensate, derivatives["dpi_dlnT_P"], derivatives["dlnn_dlnT_P"], derivatives["dlnV_dlnT_P"], derivatives["dlnV_dlnP_T"])

            velocity = np.sqrt(2 * (self.chamber["enthalpy"] - properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * properties["temperature"] * properties["gamma_s"] / properties["molar mass"])
            a_mdot = 1/(properties["density"]*velocity)
            a_at = a_mdot/at_mdot

            dlnpc_p_dlna_at = properties["gamma_s"] * velocity**2 / (velocity**2 - speed_sound**2)
            lnpc_p = lnpc_p + dlnpc_p_dlna_at * (np.log(area_ratio) - np.log(a_at))
            residual = abs(dlnpc_p_dlna_at * (np.log(area_ratio) - np.log(a_at)))

            pressure = self.chamber["pressure"]/np.exp(lnpc_p)

            gas.SP = self.throat["entropy"], pressure
            if self.assumption == 'equilibrium': 
                gas.equilibrate('SP')   
            elif self.assumption == 'frozen':
                pass
            else:
                raise ValueError("Assumption must be 'frozen' or 'equilibrium'")

        transport_prop = self.__get_transport_properties(gas, transport)

        local_prop  = { "velocity"        : velocity, 
                        "mach"            : velocity/speed_sound, 
                        "area ratio"      : area_ratio,
                        "I_sp"            : np.nan,
                        "I_vac"           : np.nan,
                        "c*"              : np.nan,
                        "C_f"             : np.nan,
                        "mole fraction"   : gas.mole_fraction_dict()}

        local_proprties = properties | transport_prop |local_prop | derivatives

        return local_proprties
    
    def property(self, location, variable):
        if location in self.engine_state.index and variable in self.engine_state.columns:
            return self.engine_state[variable][location]

In [33]:
class Engine(EngineState):     
    def __init__(self,  oxidizer, fuel, of_ratio, pressure, exit_value, gas,  size_value, condensate = None, transport=None, assumption='equilibrium', size_parameter ="thrust",
                 temp_oxidizer=None, temp_fuel=None, combustion_products=None, exit_parameter="pressure", throat_inlet_radius_ratio=1.5, temperature_wall=None):
        '''
        This class will define the properties of an rocket engine at the chamber, throat and exit 
        given the propellant, and conditions that engine is in.

        Parameters
        --------------------------------------------------
        oxidizer : string
            A string of a species name which is used to define a cantera solution representing the oxidizer.
            Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
        fuel : string
            Similar to oxidizer, this is a string of a species name which is used to define a cantera solution representing the fuel.
            Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
        of_ratio : float
            the mass ratio of oxidizer to fuel defined as mass_oxidizer / mass_fuel.
        pressure : float
            The pressure of the combustion chamber in units of Pascals (Pa).
        exit_value : float
            The exit conditions of the engine. Depending on what 'exit parameter' is
            defined to be the exit condition can be defined in the following ways:
                'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
                'area ratio': Definfed as area_exit / area_throat should be value greater than 1 TODO: add functionality
        size_value : float 
            Determines the size of the engine. depending on what 'size parameter' is defined to be. 
            exit condition can be defined in the following way
        exit_parameter : string
            A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
            or 'area ratio'. 
        temp_oxidizer : float
            A positive float value defining the temperature of the oxidizer in Kelvin (K). If the oxidizer is liquid the temperature 
            is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
        temp_fuel : float
            Similar to above. positive float value defining the temperature of the fuel in Kelvin (K). If the fuel is liquid the temperature 
            is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
        

        Attributes:
        --------------------------------------------------
        Properties : Pandas Dataframe
        
        Methods
        --------------------------------------------------

        A class for the state of a rocket engine. Given the propellants and conditions of a rocket engine this 
        class will define the properties of an engine at
        '''
        super().__init__(oxidizer, fuel, of_ratio, pressure, exit_value, gas, size_value = size_value, condensate=condensate, transport=transport, size_parameter= size_parameter,
                        exit_parameter=exit_parameter, throat_inlet_radius_ratio=throat_inlet_radius_ratio, temperature_wall=temperature_wall, assumption=assumption)
        
    def __chamber_contour(self, length_value, contraction_ratio, contraction_angle=30, nozzle_inlet_radius_ratio=0.5, 
                         throat_inlet_radius_ratio = None, length_parameter = 'characteristic length'):
        '''
        Description
        --------------------------------------------------
        will create

        Parameters
        --------------------------------------------------
        leng_value : float
            Depending on what 'size_parameter' is defined in the following ways
                'thrust' : the target thrust of the engine in units newtons
                'mass flow' : the target mass flow of the engine in units kilogram/second
                'throat diameter' : the diameter of the throat in units meter
        length_parameter : string
            a string with three possible values:
                'characteristic length' 
                'chamber length'
                'throat diameter' 

        Returns
        --------------------------------------------------
        None
        '''

        theta_c = np.radians(contraction_angle)

        r_t = self.throat["diameter"]/2
        r_c = np.sqrt(contraction_ratio * r_t**2)
        r_tin = self.throat_inlet_radius_ratio * r_t
        r_ninmax = (r_c-r_t)/(1-np.cos(theta_c)) - r_tin
        r_nin = r_ninmax * nozzle_inlet_radius_ratio

        if self.throat_inlet_radius_ratio != throat_inlet_radius_ratio and throat_inlet_radius_ratio:
            self.throat_inlet_radius_ratio = throat_inlet_radius_ratio
            self.throat.update(self._heat_flux(self.throat["temperature"],self.throat["pressure"], self.throat_transport.X, self.throat['area'], self.throat['mach']))
            self.exit.update(self._heat_flux(self.exit["temperature"],self.exit["pressure"], self.exit_transport.X, self.exit['area'], self.exit['mach']))
        
        # checks for valid inputs: 
        if not nozzle_inlet_radius_ratio <=1:
            raise ValueError("the nozzle_inlet_radius_ratio must not exceed 1. This is the ratio of the radius/ maximum possible radius")
        if r_tin *(1- np.cos(theta_c)) > r_c-r_t:
            raise ValueError("throat_inlet_radius_ratio is too high. The nozzle geometry is not possible with the given contraction ratio and contraction angle")
        

        # defines bounds for x.        
        x_tin = -r_tin * np.sin(theta_c)
        x_nin = -1/np.tan(theta_c)*(r_c-r_nin*(1-np.cos(theta_c))-
                                                    (r_t+r_tin*(1-np.cos(theta_c))))-r_tin*np.sin(theta_c)
        x_c = x_nin - r_nin * np.sin(theta_c)

        # find volume of converging section
        volume_throat_inlet = np.pi*(-(x_nin-x_c)**3/3 + (r_c-r_nin)*r_nin**2*(np.asin((x_nin-x_c)/r_nin)+1/2*np.sin(2*np.asin((x_nin-x_c)/r_nin)))+(r_nin**2+(r_c-r_nin)**2)*(x_nin-x_c))
        sub = r_t+r_tin*(1-np.cos(theta_c)-np.tan(theta_c)*np.sin(theta_c))
        volume_nozzle_line = np.pi*(np.tan(theta_c)**2*x_tin**3/3-np.tan(theta_c)*sub*x_tin**2+sub**2*x_tin) - np.pi*(np.tan(theta_c)**2*x_nin**3/3-np.tan(theta_c)*sub*x_nin**2+sub**2*x_nin) 
        volume_nozzle_inlet =  np.pi*(-x_nin**3/3+x_c*x_nin**2+(r_c-r_nin)*r_nin**2*(np.asin((x_nin-x_c)/r_nin)+np.sin(2*np.asin((x_nin-x_c)/r_nin))/2)+((r_c-r_nin)**2+r_nin**2-x_c**2)*x_nin) -np.pi*(2*x_c**3/3+((r_c-r_nin)**2+r_nin**2-x_c**2)*x_c) 
        volume_converging = volume_throat_inlet + volume_nozzle_line + volume_nozzle_inlet
        

        # find length of chamber
        if length_parameter == 'characteristic length':
            l_star = length_value            
            volume_total = self.throat["area"] * length_value

            if volume_converging > volume_total:
                raise ValueError("the converging section volume is larger than the total volume of your chamber. try increasing your L* or reducing your contraction ratio.")
            volume_chamber = volume_total - volume_converging

            l_c = volume_chamber/(np.pi*r_c**2)
        elif length_parameter == 'chamber length':
            l_c = length_value
            volume_chamber = np.pi * r_c **2 * l_c
            volume_total = volume_chamber + volume_converging
            l_star = volume_total / self.throat["area"]

        x_inj = x_c - l_c

        print(f"r_t: {r_t}, r_c: {r_c}, r_tin: {r_tin}, r_nin = {r_nin}")
        print(f"throat_inlet: {x_tin}, nozzle_inlet: {x_nin}, chamber_end: {x_c}, x_inj: {x_inj}")
        print(f"volume_throat_inlet: {volume_throat_inlet}, volume_nozzle_line: {volume_nozzle_line}, volume_nozzle_inlet: {volume_nozzle_inlet}, volume_converging: {volume_converging}, volume_chamber: {volume_chamber}, volume_total: {volume_total}")

        x_dict = {"x_inj" : 0,  "x_c" : x_c-x_inj, "x_nin" : x_nin-x_inj, "x_tin" : x_tin-x_inj, "x_t" : -x_inj}

        def chamber_contour(x): 
            x = x+x_inj
            # Defines throat
            if x == 0:
                return r_t

            # defines radius before throat
            elif x_tin <= x and x < 0:
                return -np.sqrt(r_tin**2 - x**2) + (r_tin + r_t)
            
            # defines contraction line
            elif x_nin <= x and x < x_tin:
                return -np.tan(theta_c) * (x + r_tin * np.sin(theta_c))+r_t+r_tin*(1-np.cos(theta_c))
            
            # defines radius befor nozzle
            elif x_c <= x and x < x_nin: 
                return np.sqrt(r_nin**2 - (x-x_c)**2) + (r_c - r_nin)

            # defines chamber section
            elif x_inj <= x and x < x_c:
                return r_c
            
            else: 
                raise ValueError(f"x coordinate exceeds bounds of chamber. please make sure 0 <= x <= {-x_inj}")
            
        return chamber_contour, x_dict

    def conical_contour(self, length_value, contraction_ratio, contraction_angle=30, nozzle_inlet_radius_ratio=0.5, throat_inlet_radius_ratio = None, 
                                length_parameter = 'characteristic length', throat_outlet_radius_ratio=0.382, expansion_angle=15, fidelity = 500):
        
        chamber_contour, chamber_x_dict = self.__chamber_contour(length_value, contraction_ratio, contraction_angle, nozzle_inlet_radius_ratio, throat_inlet_radius_ratio, length_parameter = 'characteristic length')

        theta_e = np.radians(expansion_angle)
        r_t = self.throat["diameter"]/2
        r_tout = throat_outlet_radius_ratio * r_t

        x_tout = r_tout * np.sin(theta_e) + chamber_x_dict["x_t"]
        x_e = (self.exit["diameter"]/2-r_t-r_tout*(1-np.cos(theta_e)))/np.tan(theta_e)+r_tout*np.sin(theta_e) + chamber_x_dict["x_t"]
        
        nozzle_x_dict = {"x_tout" : x_tout, "x_e" : x_e}

        x_dict = chamber_x_dict | nozzle_x_dict
        
        def contour(x):
            if 0 <= x and x <= x_dict["x_t"]: 
                return chamber_contour(x)
            elif x_dict["x_t"] < x <= x_dict["x_tout"]:
                x = x-x_dict["x_t"]
                return -np.sqrt(-x**2+r_tout**2) + r_t +r_tout
            elif x_dict["x_tout"] < x and x <= x_dict["x_e"]:
                x = x-x_dict["x_t"]
                return np.tan(theta_e)*(x-r_tout*np.sin(theta_e))+r_t+r_tout*(1-np.cos(theta_e))
            else:
                raise ValueError(f"x coordinate exceeds bounds of nozzle. please make sure {x_dict['x_inj']} < x <= {x_dict['x_e']}")

        # x_coords = np.linspace(x_dict["x_inj"], x_dict["x_e"], fidelity)
        x_coords = np.sort(np.append(np.linspace(x_dict["x_inj"], x_dict["x_e"], fidelity), list(x_dict.values())))
        contour_coords = []
        for x in x_coords: 
            contour_coords.append([x, contour(x)])

        print(f"r_t: {r_t}, r_tout: {r_tout}")

        self.contour = contour
        self.x_dict = x_dict
        self.contour_coords = contour_coords

        return contour_coords

    def properties_along_contour(self, fidelity):
        stations = np.hstack((0, np.linspace(self.x_dict["x_c"], self.x_dict["x_e"], fidelity, endpoint = True)))
        
        if self.x_dict["x_t"] not in  stations: 
            idx = np.searchsorted(stations, self.x_dict["x_t"])
            stations = np.insert(stations, idx, self.x_dict["x_t"])
        
        self._gas.TPX = self.chamber_gas.T, self.chamber_gas.P, self.chamber_gas.X

        list_station_properties = []

        for station in stations:
            area_ratio = np.pi*self.contour(station)**2 / self.throat["area"]
            if 0 <= station and station < self.x_dict['x_t']:   
                station_properties = self.state_at_area(self._gas, self._condensate, self._transport, area_ratio, speed = "subsonic")
                station_size = {'thrust' :      np.nan,
                                'mass flow' :   self.throat['mass flow'],
                                'area' :        self.throat['area'] * area_ratio,
                                'diameter' :    self.contour(station)*2}
                
                station_heat = self._heat_flux(station_properties['temperature'], station_properties['pressure'], self._transport.X, 
                                                          station_size['area'], station_properties['mach'], viscosity_exponent = 0.6)

                station_properties.update(station_size | station_heat)
            
            elif station == self.x_dict['x_t']:
                station_properties = self.throat

            elif self.x_dict['x_t'] < station and station < self.x_dict['x_e']:
                station_properties = self.state_at_area(self._gas, self._condensate, self._transport, area_ratio, speed = "supersonic")
                station_size = {'thrust' :      np.nan,
                                'mass flow' :   self.throat['mass flow'],
                                'area' :        self.throat['area'] * area_ratio,
                                'diameter' :    self.contour(station)*2}
                
                station_heat = self._heat_flux(station_properties['temperature'], station_properties['pressure'], self._transport.X, 
                                                          station_size['area'], station_properties['mach'])

                station_properties.update(station_size | station_heat)
            
            elif station == self.x_dict['x_e']:
                station_properties = self.exit
        
            else:
                raise ValueError(f"Invalid station: station must be greater than 0 (injector face) and less that {self.x_dict['x_e']} (nozzle exit) in order to be valid.")
            
            list_station_properties.append(station_properties)

        self.property_contour = pd.DataFrame(list_station_properties, stations)

                

        return self.property_contour 

    def parabolic_contour():
        return
    
    def bell_contour():
        return
    
    def channel_design():
       return 

In [34]:
def ranged_sim_rocketcow(oxidizer, fuel, of_arr, p_arr, exit_value, gas, condensate = None, transport = None, size_value=None, size_parameter ="thrust", exit_parameter="pressure"):
    state_list = []
    start_time = time.time()
    for pressure in p_arr:
        for of_ratio in of_arr:
            state = Engine(oxidizer, fuel, of_ratio, pressure, exit_value, gas, size_value, condensate=condensate, transport=transport, size_parameter=size_parameter,
                           exit_parameter=exit_parameter)
            t1 = time.time()
            state_list.append(state())
            t2 = time.time() -t1
    iter_time = time.time() - start_time

    states = pd.concat(state_list, keys = list(range(len(state_list))))

    concat_time = time.time()- start_time-iter_time

    print(f"-----TOTAL-----\ntotal time: {time.time()-start_time}\niter time: {iter_time}\nconcat time: {concat_time},\ntime per case: {(time.time()-start_time)/(len(p_arr)*len(of_arr))}\nnumber of cases: {(len(p_arr)*len(of_arr))}")
    return states

In [35]:
def chemistry_initializer(oxidizer, fuel, temp_oxidizer=None, temp_fuel=None, combustion_products=None):
    propellant_array = [['oxidizer', oxidizer, temp_oxidizer],['fuel', fuel, temp_fuel]]
    reactants_all   = {S.name: S for S in ct.Species.list_from_file('chem_prop/reactants.yaml')}
    gaseous_all     = {S.name: S for S in ct.Species.list_from_file('chem_prop/gaseous_products.yaml')}
    condensed_all   = {S.name: S for S in ct.Species.list_from_file('chem_prop/condensed_products.yaml')}
    propellant = {}
    propellant_elements = []

    def transport_species(gaseous_species):
        transport_species = [sp for sp in gaseous_species.values() if sp.transport is not None]
        if not transport_species:
            return None
        transport = ct.Solution(thermo="ideal-gas", transport = "mixture-averaged", species = transport_species)
        transport.transport_model = "mixture-averaged"
        return transport
    
    # initialize propellants
    for type, name, temp in propellant_array:
        if name in reactants_all:
            if temp is None:
                if reactants_all[name].thermo.input_data['model'] == 'constant-cp':
                    temp = reactants_all[name].thermo.input_data['T0']
                else:
                    temp= 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[reactants_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')
        
        elif name in gaseous_all:
            if temp is None:
                temp = 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[gaseous_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')

        
        elif name in condensed_all:
            if temp is None:
                temp = 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[condensed_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')

        else:
            raise ValueError(f"{name} does not exist in the thermal database.")    
        
        propellant_elements.extend(propellant[type].element_names)

    # ensures propellant elements are unique
    propellant_elements = list(set(propellant_elements))

    if combustion_products == None: 
        condensed_species = {k: v for k, v in condensed_all.items() if set(v.composition.keys()).issubset(propellant_elements)}
        gaseous_species = {k: v for k, v in gaseous_all.items() if set(v.composition.keys()).issubset(propellant_elements) and ('+' not in k and '-' not in k)}
        transport = transport_species(gaseous_species)
    else: 
        # checks if designated combustion products exist in the thermal database. if it does then filters the data
        if set(combustion_products).issubset(set(condensed_all.keys()) | set(gaseous_all.keys())):
            condensed_species = {k: v for k, v in condensed_all.items() if k in combustion_products and set(v.composition.keys()).issubset(propellant_elements)}
            gaseous_species = {k: v for k, v in gaseous_all.items() if k in combustion_products and set(v.composition.keys()).issubset(propellant_elements) and ('+' not in k and '-' not in k)}
            transport = transport_species(gaseous_species)
        else:
            invalid_species = set(combustion_products) - (set(condensed_all.keys()) | set(gaseous_all.keys()))
            raise ValueError(f"the following species {invalid_species} do not exist in the thermal database.")   
    
    if condensed_species:
        condensate = ct.Solution(thermo='ideal-gas', species=list(condensed_species.values()))
    else: 
        condensate = None
    if gaseous_species:
        gas =  ct.Solution(thermo='ideal-gas', species=list(gaseous_species.values()))
    else: 
        raise ValueError("No valid gas phase combustion products of given propellants. Consider changing propllants or inputted combustion products")        

    return propellant['oxidizer'], propellant['fuel'], gas, condensate, transport

In [36]:
of = 1.5
p = 20*1e5
Ox = 'O2(L)' 
f = 'CH4(L)'
p_atm = 1.01325*1e5


oxidizer, fuel, gas, condensate, transport = chemistry_initializer('O2(L)', "CH4(L)")

                # (oxidizer, fuel, of_ratio, pressure, exit_value, gas, condensate = None, size_value=None, size_parameter ="thrust",  exit_parameter="pressure")  
example = EngineState(oxidizer, fuel, of, p, p_atm, gas, exit_parameter="pressure", transport=transport, assumption='frozen')

example()

-----
total time: 0.16857385635375977
chemistry initialization: 0.16053462028503418
 chamber properties: 0.006010770797729492
throat time: 0.0
exit time: 0.0
sizing time: 0.0
df making: 0.0020284652709960938 


,of ratio,pressure,temperature,density,specific volume,enthalpy,internal energy,gibbs,entropy,molar mass,c_p,c_v,gamma,gamma_s,speed sound,viscosity,thermal conductivity,prandtl number,velocity,mach,area ratio,I_sp,I_vac,c*,C_f,mole fraction,dpi_dlnT_P,dlnn_dlnT_P,dpi_dlnP_T,dlnn_dlnP_T,dlnV_dlnT_P,dlnV_dlnP_T
chamber,1.5,2.000000e+06,1668.337266,1.927922,0.518693,-2.468217e+06,-3.505603e+06,-2.759732e+07,15062.365423,13.371421,2750.166732,2128.358281,1.292154,1.292154,1157.783561,0.000054,0.307308,0.480774,0.000000,0.000000,NaN,NaN,NaN,1529.678192,NaN,"{'CH4': 8.783943068289706e-05, 'CO': 0.3078037...","[nan, nan, nan]",0,"[nan, nan, nan]",0,1,-1
throat,1.5,1.090044e+06,1451.673021,1.207588,0.828097,-3.056193e+06,-3.958855e+06,-2.492182e+07,15062.365423,13.371421,2675.617784,2053.809333,1.302759,1.302759,1084.412933,0.000049,0.273446,0.477277,1084.412933,1.000000,NaN,NaN,NaN,NaN,NaN,"{'CH4': 8.783943068289708e-05, 'CO': 0.3078037...","[nan, nan, nan]",0,"[nan, nan, nan]",0,1,-1
exit,1.5,1.013250e+05,810.214634,0.201122,4.972103,-4.689331e+06,-5.193129e+06,-1.689308e+07,15062.365423,13.371421,2411.206904,1789.398453,1.347496,1.347496,823.933312,0.000032,0.168844,0.459460,2107.659201,2.558046,3.08925,214.921426,239.295922,NaN,1.377845,"{'CH4': 8.783943068289706e-05, 'CO': 0.3078037...","[nan, nan, nan]",0,"[nan, nan, nan]",0,1,-1


In [37]:
of = 1.5
p = 20*1e5
Ox = 'O2(L)' 
f = 'CH4(L)'
p_atm = 1.01325*1e5


oxidizer, fuel, gas, condensate, transport = chemistry_initializer('O2(L)', "CH4(L)")

                # (oxidizer, fuel, of_ratio, pressure, exit_value, gas, condensate = None, size_value=None, size_parameter ="thrust",  exit_parameter="pressure")  
example = EngineState(oxidizer, fuel, of, p, 3.08925, gas, exit_parameter="area ratio", transport=transport, assumption='frozen')

example()

-----
total time: 0.1855297088623047
chemistry initialization: 0.16681909561157227
 chamber properties: 0.01571202278137207
throat time: 0.0019996166229248047
exit time: 0.0
sizing time: 0.0
df making: 0.0009989738464355469 


,of ratio,pressure,temperature,density,specific volume,enthalpy,internal energy,gibbs,entropy,molar mass,c_p,c_v,gamma,gamma_s,speed sound,viscosity,thermal conductivity,prandtl number,velocity,mach,area ratio,I_sp,I_vac,c*,C_f,mole fraction,dpi_dlnT_P,dlnn_dlnT_P,dpi_dlnP_T,dlnn_dlnP_T,dlnV_dlnT_P,dlnV_dlnP_T
chamber,1.5,2.000000e+06,1668.337266,1.927922,0.518693,-2.468217e+06,-3.505603e+06,-2.759732e+07,15062.365423,13.371421,2750.166732,2128.358281,1.292154,1.292154,1157.783561,0.000054,0.307308,0.480774,0.000000,0.000000,NaN,NaN,NaN,1529.678192,NaN,"{'CH4': 8.783943068289706e-05, 'CO': 0.3078037...","[nan, nan, nan]",0,"[nan, nan, nan]",0,1,-1
throat,1.5,1.090044e+06,1451.673021,1.207588,0.828097,-3.056193e+06,-3.958855e+06,-2.492182e+07,15062.365423,13.371421,2675.617784,2053.809333,1.302759,1.302759,1084.412933,0.000049,0.273446,0.477277,1084.412933,1.000000,NaN,NaN,NaN,NaN,NaN,"{'CH4': 8.783943068289708e-05, 'CO': 0.3078037...","[nan, nan, nan]",0,"[nan, nan, nan]",0,1,-1
exit,1.5,1.013240e+05,810.208530,0.201121,4.972127,-4.689353e+06,-5.193149e+06,-1.689301e+07,15062.365423,13.371382,2411.207230,1789.396972,1.347497,1.347497,823.931802,0.000032,0.168843,0.459460,2107.669700,2.558063,3.08925,214.922496,239.296758,NaN,1.377852,"{'CH4': 8.783962511866764e-05, 'CO': 0.3078044...","[nan, nan, nan]",0,"[nan, nan, nan]",0,1,-1


In [38]:
of = 6
p = 1.823e7
Ox = 'O2(L)' 
f = 'H2(L)'
# p_atm = 1.01325*1e5
area_ratio = ureg.convert(12, "in", "m")**2 / ureg.convert(5.4417, "in", "m")**2  
throat_diameter = ureg.convert(5.4417, "in", "m")*2 
throat_inlet_radius_ratio = 0.494

oxidizer, fuel, gas, condensate, transport = chemistry_initializer(Ox, f)
test_engine = Engine(oxidizer, fuel, of, p, area_ratio, gas, exit_parameter="area ratio", transport=transport, size_value=throat_diameter, 
                     temperature_wall=866.6, size_parameter="throat diameter", throat_inlet_radius_ratio=throat_inlet_radius_ratio, assumption = 'frozen')
test_engine()

-----
total time: 0.01772451400756836
chemistry initialization: 0.0149078369140625
 chamber properties: 0.0013096332550048828
throat time: 0.0
exit time: 0.0
sizing time: 0.0015070438385009766
df making: 0.0 


,of ratio,pressure,temperature,density,specific volume,enthalpy,internal energy,gibbs,entropy,molar mass,c_p,c_v,gamma,gamma_s,speed sound,viscosity,thermal conductivity,prandtl number,velocity,mach,area ratio,I_sp,I_vac,c*,C_f,mole fraction,dpi_dlnT_P,dlnn_dlnT_P,dpi_dlnP_T,dlnn_dlnP_T,dlnV_dlnT_P,dlnV_dlnP_T,thrust,mass flow,area,diameter,heat transfer coefficient,heat flux
chamber,6,1.823000e+07,3584.934088,8.315427,0.120258,-9.862790e+05,-3.178590e+06,-6.283498e+07,17252.396889,13.596065,3793.497533,3181.963059,1.192188,1.192188,1616.677507,0.000104,0.636790,0.621019,0.000000,0.000000,NaN,NaN,NaN,2288.443648,NaN,"{'H': 0.02646784850519155, 'H2': 0.24747398083...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,inf,inf,NaN,NaN
throat,6,1.030512e+07,3267.862631,5.156659,0.193924,-2.180808e+06,-4.179219e+06,-5.855927e+07,17252.396889,13.596065,3739.819325,3128.284852,1.195486,1.195486,1545.662011,0.000097,0.586872,0.620939,1545.662011,1.000000,NaN,NaN,NaN,NaN,NaN,"{'H': 0.026467848505191554, 'H2': 0.2474739808...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.060019,0.276438,51385.556502,1.372509e+08
exit,6,5.936573e+05,2004.042339,0.484404,2.064392,-6.710661e+06,-7.936202e+06,-4.128519e+07,17252.396889,13.596065,3375.610727,2764.076244,1.221244,1.221244,1223.390466,0.000067,0.369145,0.610381,3383.602159,2.765758,4.862877,345.031398,381.965528,NaN,1.47856,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,1.618634e+06,478.37606,0.291864,0.609600,9008.610177,2.277409e+07


In [39]:
of_arr = np.arange(1, 5, .5)
p_arr = np.arange(Q(400, 'psi').to_base_units().magnitude, Q(1001, 'psi').to_base_units().magnitude, Q(100, 'psi').to_base_units().magnitude)
exit_value = 8 #in^2
rocketcow_results = ranged_sim_rocketcow(oxidizer, fuel, of_arr, p_arr, exit_value, gas, condensate=condensate, transport=transport, size_value=1000, size_parameter ="thrust", exit_parameter="area ratio")

rocketcow_results

temperature_wall is not defined. If you want heat transfer coeff. try initiating class with temperature wall defined, or setting [object_name].temperature_wall to chosen value.
temperature_wall is not defined. If you want heat transfer coeff. try initiating class with temperature wall defined, or setting [object_name].temperature_wall to chosen value.
-----
total time: 0.018617630004882812
chemistry initialization: 0.00950932502746582
 chamber properties: 0.0
throat time: 0.0033721923828125
exit time: 0.002498626708984375
sizing time: 0.000858306884765625
df making: 0.002379179000854492 
temperature_wall is not defined. If you want heat transfer coeff. try initiating class with temperature wall defined, or setting [object_name].temperature_wall to chosen value.
temperature_wall is not defined. If you want heat transfer coeff. try initiating class with temperature wall defined, or setting [object_name].temperature_wall to chosen value.
-----
total time: 0.012973785400390625
chemistry in

of ratio      pressure  temperature   density  specific volume  \
0  chamber       1.0  2.757903e+06   977.565760  1.368105         0.730938   
   throat        1.0  1.471254e+06   826.648579  0.863084         1.158636   
   exit          1.0  2.926685e+04   279.270372  0.050820        19.677164   
1  chamber       1.5  2.757903e+06  1411.995145  1.183973         0.844614   
   throat        1.5  1.489471e+06  1214.867967  0.743189         1.345553   
...              ...           ...          ...       ...              ...   
54 throat        4.0  3.891284e+06  2694.838762  1.743471         0.573568   
   exit          4.0  1.045840e+05  1353.867199  0.093650        10.678065   
55 chamber       4.5  6.894757e+06  3132.412253  2.893447         0.345609   
   throat        4.5  3.918190e+06  2895.859319  1.788731         0.559055   
   exit          4.5  1.100331e+05  1546.039148  0.094911        10.536182   

                enthalpy  internal energy         gibbs       entropy  \
0  chamber -2.437929e+06    -4.453785e+06 -3.869921e+07  37093.449446   
   throat  -3.604421e+06    -5.309068e+06 -3.426767e+07  37093.449446   
   exit    -7.695000e+06    -8.270889e+06 -1.805410e+07  37093.449447   
1  chamber -2.031467e+06    -4.360829e+06 -4.878096e+07  33108.822310   
   throat  -3.364355e+06    -5.368517e+06 -4.358720e+07  33108.822310   
...                  ...              ...           ...           ...   
54 throat  -2.556805e+06    -4.788722e+06 -6.044096e+07  21479.637627   
   exit    -8.465839e+06    -9.582594e+06 -3.754642e+07  21479.637627   
55 chamber -1.144641e+06    -3.527528e+06 -6.493706e+07  20365.268163   
   throat  -2.436531e+06    -4.627016e+06 -6.141148e+07  20365.268163   
   exit    -8.313971e+06    -9.473300e+06 -3.979947e+07  20365.268161   

            molar mass          c_p          c_v     gamma   gamma_s  \
0  chamber    4.032000  7810.337287  5748.218570  1.358740  1.358740   
   throat     4.032000  7656.015532  5593.896827  1.368637  1.368637   
   exit       4.032000  7265.133918  5203.015214  1.396331  1.396331   
1  chamber    5.039997  6875.275281  5225.548457  1.315704  1.315704   
   throat     5.040000  6646.939432  4997.242209  1.330122  1.330122   
...                ...          ...          ...       ...       ...   
54 throat    10.038962  5369.482877  4468.400076  1.201657  1.199204   
   exit      10.079812  3843.537344  3018.645342  1.273266  1.273265   
55 chamber   10.929735  6442.430229  5461.493296  1.179610  1.171053   
   throat    10.991862  5739.691245  4844.706128  1.184735  1.179547   
   exit      11.087871  3718.251760  2968.119297  1.252730  1.252724   

            speed sound  viscosity  thermal conductivity  prandtl number  \
0  chamber  1655.000310   0.000026              0.349819        0.574607   
   throat   1527.430539   0.000023              0.303921        0.570305   
   exit      896.733774   0.000010              0.128534        0.539481   
1  chamber  1750.642978   0.000037              0.446900        0.570550   
   throat   1632.721189   0.000033              0.390619        0.562117   
...                 ...        ...                   ...             ...   
54 throat   1636.008935   0.000077              0.602761        0.585944   
   exit     1192.444908   0.000044              0.308411        0.552851   
55 chamber  1670.475273   0.000089              0.659390        0.591599   
   throat   1607.414240   0.000084              0.611489        0.592704   
   exit     1205.121627   0.000051              0.330112        0.570482   

               velocity      mach  area ratio        I_sp       I_vac  \
0  chamber     0.000000  0.000000         NaN         NaN         NaN   
   throat   1527.430539  1.000000         NaN         NaN         NaN   
   exit     3242.551879  3.615958         8.0  330.648272  348.758792   
1  chamber     0.000000  0.000000         NaN         NaN         NaN   
   throat   1632.721189  1.000000         NaN         N

In [40]:
ureg.convert(0.3665850979211999, "m", "in")
# ureg.convert(36, "in", "m")*test_engine.throat["area"]      #/((1.643* ureg.convert(5.4417, "in", "m"))**2*np.pi)

14.432484170125981

In [41]:
Lstar = ureg.convert(36, "in", "m")
contraction_ratio = (1.643* ureg.convert(5.4417, "in", "m"))**2 /(ureg.convert(5.4417, "in", "m"))**2

contour_list = test_engine.conical_contour(Lstar, contraction_ratio, length_parameter= 'characteristic length', fidelity=100)
contour_df = pd.DataFrame(contour_list, columns=["x", "y"])
fig = px.line(contour_df, x="x", y="y")
fig.update_yaxes(range=[0,None])
# print(contour_df)

r_t: 0.13821918, r_c: 0.22709411274, r_tin: 0.06828027492, r_nin = 0.2975456270449468
throat_inlet: -0.03414013745999999, nozzle_inlet: -0.10318584050436785, chamber_end: -0.25195865402684126, x_inj: -0.2875704987636982
volume_throat_inlet: 0.021505729909219035, volume_nozzle_line: 0.006099890379830282, volume_nozzle_inlet: 0.021505729909219028, volume_converging: 0.049111350198268344, volume_chamber: 0.005769738033357265, volume_total: 0.05488108823162561
r_t: 0.13821918, r_tout: 0.05279972676


In [ ]:
start = time.time()
contour = test_engine.properties_along_contour(100)
time.time()- start

0.008672714233398438

In [46]:
contour

,of ratio,pressure,temperature,density,specific volume,enthalpy,internal energy,gibbs,entropy,molar mass,c_p,c_v,gamma,gamma_s,speed sound,viscosity,thermal conductivity,prandtl number,velocity,mach,area ratio,I_sp,I_vac,c*,C_f,mole fraction,dpi_dlnT_P,dlnn_dlnT_P,dpi_dlnP_T,dlnn_dlnP_T,dlnV_dlnT_P,dlnV_dlnP_T,thrust,mass flow,area,diameter,heat transfer coefficient,heat flux
0.000000,6,1.768545e+07,3567.444525,8.106586,0.123356,-1.052602e+06,-3.234217e+06,-6.259957e+07,17252.396889,13.596065,3790.750369,3179.215886,1.192354,1.192354,1612.841437,0.000104,0.634086,0.621017,364.205489,0.225816,2.699449,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.162017,0.454188,21532.544252,5.847607e+07
0.035612,6,1.768545e+07,3567.444525,8.106586,0.123356,-1.052602e+06,-3.234217e+06,-6.259957e+07,17252.396889,13.596065,3790.750369,3179.215886,1.192354,1.192354,1612.841437,0.000104,0.634086,0.621017,364.205489,0.225816,2.699449,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.162017,0.454188,21532.544252,5.847607e+07
0.044507,6,1.768412e+07,3567.401282,8.106075,0.123364,-1.052766e+06,-3.234355e+06,-6.259899e+07,17252.396889,13.596065,3790.743549,3179.209066,1.192354,1.192354,1612.831941,0.000104,0.634080,0.621017,364.655302,0.226096,2.696288,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.161828,0.453922,21555.188721,5.853743e+07
0.053402,6,1.768011e+07,3567.270602,8.104532,0.123388,-1.053261e+06,-3.234770e+06,-6.259723e+07,17252.396889,13.596065,3790.722939,3179.188456,1.192356,1.192356,1612.803244,0.000104,0.634059,0.621017,366.011247,0.226941,2.686809,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.161259,0.453124,21623.405160,5.872226e+07
0.062297,6,1.767332e+07,3567.049593,8.101922,0.123428,-1.054099e+06,-3.235473e+06,-6.259425e+07,17252.396889,13.596065,3790.688081,3179.153597,1.192358,1.192358,1612.754709,0.000104,0.634025,0.621017,368.293082,0.228363,2.671019,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.160311,0.451790,21738.051405,5.903288e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.880630,6,6.492292e+05,2036.725559,0.521248,1.918472,-6.600102e+06,-7.845630e+06,-4.173850e+07,17252.396889,13.596065,3389.780664,2778.246181,1.220115,1.220115,1232.756123,0.000068,0.375133,0.611129,3350.768112,2.718111,4.563432,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.273891,0.590533,11427.804088,2.847274e+07
0.889525,6,6.346726e+05,2028.406049,0.511651,1.954457,-6.628289e+06,-7.868729e+06,-4.162316e+07,17252.396889,13.596065,3386.205917,2774.671434,1.220399,1.220399,1230.378758,0.000067,0.373610,0.610942,3359.169519,2.730191,4.637401,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.278331,0.595300,11249.560768,2.801512e+07
0.898420,6,6.205705e+05,2020.187138,0.502318,1.990772,-6.656105e+06,-7.891519e+06,-4.150918e+07,17252.396889,13.596065,3382.652873,2771.118390,1.220681,1.220681,1228.025691,0.000067,0.372105,0.610755,3367.440057,2.742158,4.711965,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.37606,0.282806,0.600066,11075.474894,2.756843e+07
0.907315,6,6.069045e+05,2012.066614,0.493239,2.027416,-6.683560e+06,-7.914008e+06,-4.139653e+07,17252.396889,13.596065,3379.121238,2767.586755,1.220963,1.220963,1225.696422,0.000067,0.370617,0.610568,3375.583167,2.754012,4.787124,NaN,NaN,NaN,NaN,"{'H': 0.026467848675226505, 'H2': 0.2474739824...","[nan, nan]",0,"[nan, nan]",0,1,-1,NaN,478.3

In [87]:
fig = go.Figure()
fig.update_layout(xaxis=dict(title = dict(text="Axial distance (m)")), 
                  title_text="Properties along Engine contour")
properties = [['pressure', '(Pa)'], ['temperature', '(K)'], ['velocity', '(m/s)'], ['heat transfer coefficient', '(W/m^2/K)']]

fig.add_trace(go.Scatter(x = contour.index, y = contour['diameter']/2, name= f"radius (m)"))

for i, prop in enumerate(properties): 
    fig.add_trace(go.Scatter(x = contour.index, y = contour[prop[0]], name= f"{prop[0]} {prop[1]}", yaxis=f"y{i+2}"))

fig.update_layout(yaxis=dict(title=dict(text="Radius (m)"),side="right", rangemode='tozero'),
                  yaxis2=dict(title=dict(text="Pressure (Pa)"),overlaying="y"),
                  yaxis3=dict(title=dict(text="Temperature (K)"),anchor="free",overlaying="y",autoshift=True),
                  yaxis4=dict(title=dict(text="Velocity (m/s)"),anchor="free",overlaying="y",autoshift=True),
                  yaxis5=dict(title=dict(text='heat transfercoefficient (W/m^2/K)'),anchor="free",overlaying="y",autoshift=True))
fig.show()

In [ ]:
def CEA_wrap_ranged_sim(oxidizer, fuel, of_arr, p_arr, p_e=1.01325, assumption = 'Equilibrium'):

    f = CEA.Fuel(fuel, sum(CEA.ThermoInterface[fuel].temp_ranges[0])/2)
    o = CEA.Oxidizer(oxidizer, sum(CEA.ThermoInterface[oxidizer].temp_ranges[0])/2)
    problem = CEA.RocketProblem(materials=[o,f], pressure_units='bar')
    collector = CEA.DataCollector(  "o_f", "p", "t_p", "c_p", "t", "t_t", "c_t", "h", "t_h", "c_h", "rho", "t_rho", "c_rho", "son", "t_son", "c_son", "visc", "t_visc", "c_visc", "cond", "t_cond", "c_cond", 
                                    "pran", "t_pran", "c_pran", "mw", "t_mw", "c_mw", "m", "t_m", "c_m", "condensed", "t_condensed", "c_condensed", "cp", "t_cp", "c_cp", "gammas", "t_gammas", "c_gammas",
                                    "gamma", "t_gamma", "c_gamma", "isp", "t_isp", "ivac", "t_ivac", "cf", "t_cf", "cstar", "mach", "phi", "ae", "t_ae", "pip", "t_pip", "prod_c", "prod_t", "prod_e",
                                    "dLV_dLP_t", "t_dLV_dLP_t", "c_dLV_dLP_t", "dLV_dLT_p", "t_dLV_dLT_p", "c_dLV_dLT_p")
    for p in p_arr: 
        problem.set_pressure(p)
        problem.set_pip(p/p_e)
        for of in of_arr: 
            f.wt = 1
            o.wt = of
            problem.set_absolute_o_f()
            collector.add_data(problem.run())
    collector["prod_c"] = list(map(dict, collector['prod_c']))
    collector["prod_t"] = list(map(dict, collector['prod_t']))
    collector["prod_e"] = list(map(dict, collector['prod_e']))
    
    chamber, throat, exit = {}, {}, {}
    chamber['of ratio'], throat['of ratio'], exit['of ratio'] = collector['o_f'], collector['o_f'], collector['o_f']
    chamber['pressure'], throat['pressure'], exit['pressure'] = [x*1e5 for x in collector['c_p']], [x*1e5 for x in collector['t_p']], [x*1e5 for x in collector['p']]
    chamber['temperature'], throat['temperature'], exit['temperature'] = collector['c_t'], collector['t_t'], collector['t']
    chamber['density'], throat['density'], exit['density'] = collector['c_rho'], collector['t_rho'], collector['rho']
    chamber['specific volume'], throat['specific volume'], exit['specific volume'] = [1/x for x in collector['c_rho']], [1/x for x in collector['t_rho']], [1/x for x in collector['rho']]
    chamber['enthalpy'], throat['enthalpy'], exit['enthalpy'] = collector['c_h'], collector['t_h'], collector['h']
    chamber['internal energy'], throat['internal energy'], exit['internal energy'] = 0,0,0
    chamber['gibbs'], throat['gibbs'], exit['gibbs'] = 0,0,0
    chamber['entropy'], throat['entropy'], exit['entropy'] = 0,0,0
    chamber['molar mass'], throat['molar mass'], exit['molar mass'] = collector['c_mw'], collector['t_mw'], collector['mw']
    chamber['c_p'], throat['c_p'], exit['c_p'] = collector['c_cp'], collector['t_cp'], collector['cp']
    chamber['c_v'], throat['c_v'], exit['c_v'] = [x/y for x, y in zip(collector['c_cp'], collector['c_gamma'])], [x/y for x, y in zip(collector['t_cp'],collector['t_gamma'])], [x/y for x, y in zip(collector['cp'],collector['gamma'])]
    chamber['gamma'], throat['gamma'], exit['gamma'] = collector['c_gamma'], collector['t_gamma'], collector['gamma']
    chamber['gamma_s'], throat['gamma_s'], exit['gamma_s'] = collector['c_gammas'], collector['t_gammas'], collector['gammas']
    chamber['speed sound'], throat['speed sound'], exit['speed sound'] = collector['c_son'], collector['t_son'], collector['son']
    chamber['viscosity'], throat['viscosity'], exit['viscosity'] = collector['c_visc'], collector['t_visc'], collector['visc']
    chamber['thermal conductivity'], throat['thermal conductivity'], exit['thermal conductivity'] = collector['c_cond'], collector['t_cond'], collector['cond']
    chamber['prandtl number'], throat['prandtl number'], exit['prandtl number'] = collector['c_pran'], collector['t_pran'], collector['pran']
    chamber['velocity'], throat['velocity'], exit['velocity'] = 0, collector['t_son'], [x*y for x, y in zip(collector['mach'],collector['son'])]
    chamber['mach'], throat['mach'], exit['mach'] = 0, 1, collector['mach']
    chamber['area ratio'], throat['area ratio'], exit['area ratio'] = np.nan, collector['t_ae'], collector['ae']
    chamber['I_sp'], throat['I_sp'], exit['I_sp'] = 0, collector['t_isp'], collector['isp']
    chamber['I_vac'], throat['I_vac'], exit['I_vac'] = 0, collector['t_ivac'], collector['ivac']
    chamber['c*'], throat['c*'], exit['c*'] = collector['cstar'], 0, 0
    chamber['C_f'], throat['C_f'], exit['C_f'] = 0, collector['t_cf'], collector['cf']
    chamber['mole fraction'], throat['mole fraction'], exit['mole fraction'] = collector['prod_c'], collector['prod_t'], collector['prod_e']
    chamber['dlnV_dlnP_T'], throat['dlnV_dlnP_T'], exit['dlnV_dlnP_T'] = collector['c_dLV_dLP_t'], collector['t_dLV_dLP_t'], collector['dLV_dLP_t']
    chamber['dlnV_dlnT_P'], throat['dlnV_dlnT_P'], exit['dlnV_dlnT_P'] = collector['c_dLV_dLT_p'], collector['t_dLV_dLT_p'], collector['dLV_dLT_p']

    stacked = pd.concat([pd.DataFrame(chamber), pd.DataFrame(throat), pd.DataFrame(exit)], keys=['chamber', 'throat', 'exit'])
    df = stacked.swaplevel(0, 1).sort_index(level=0, sort_remaining=False)
    
    df['enthalpy'] = df['enthalpy'] * 1e3
    df['c_p'] = df['c_p'] * 1e3
    df['c_v'] = df['c_v'] * 1e3


    return df

In [ ]:
of_arr = np.arange(1, 4.01, .5).tolist()
p_arr = np.arange(20, 40, 5).tolist()
Ox = 'O2(L)' 
f = 'CH4(L)'
p_atm = 1.01325

oxidizer, fuel, gas, condensate, transport = chemistry_initializer(Ox, f)

rocketcow_results = ranged_sim_rocketcow(oxidizer, fuel, of_arr, (np.array(p_arr)*1e5).tolist(), p_atm*1e5, gas, transport=transport).drop(columns=['dpi_dlnT_P', 'dlnn_dlnT_P', 'dpi_dlnP_T', 'dlnn_dlnP_T'])
CEA_results = CEA_wrap_ranged_sim(Ox, f, of_arr, p_arr, p_atm)
rocketcow_composition = rocketcow_results.pop('mole fraction')
CEA_composition = CEA_results.pop('mole fraction')

-----
total time: 0.18101716041564941
chemistry initialization: 0.16449666023254395
 chamber properties: 0.012804508209228516
throat time: 0.003715991973876953
exit time: 0.0
sizing time: 0.0
df making: 0.0 
-----
total time: 0.21338701248168945
chemistry initialization: 0.19713282585144043
 chamber properties: 0.010902881622314453
throat time: 0.002772092819213867
exit time: 0.002579212188720703
sizing time: 0.0
df making: 0.0 
-----
total time: 0.1796720027923584
chemistry initialization: 0.15658998489379883
 chamber properties: 0.016261816024780273
throat time: 0.004820585250854492
exit time: 0.0
sizing time: 0.0
df making: 0.0019996166229248047 
-----
total time: 0.16802501678466797
chemistry initialization: 0.15171265602111816
 chamber properties: 0.0
throat time: 0.016312360763549805
exit time: 0.0
sizing time: 0.0
df making: 0.0 
-----
total time: 0.18155741691589355
chemistry initialization: 0.1551682949066162
 chamber properties: 0.01733851432800293
throat time: 0.0
exit time:

In [ ]:
pd.set_option('display.max_rows', None)
delta = np.divide(np.subtract(rocketcow_results, CEA_results),CEA_results)*100
compare = pd.concat({'Rocket Cow': rocketcow_results, 'CEA': CEA_results, "delta %": delta}, axis=1)

idx = pd.IndexSlice
compare.sort_index(axis=1, level=1).loc[idx[1:5,:], idx[:,["thermal conductivity", "viscosity", "prandtl number"]]]
#.loc[idx[1:5,:], idx['CEA',["thermal conductivity", "viscosity", "prandtl number"]]]
#.loc[idx[:,:], idx[:,["thermal conductivity", "viscosity", "prandtl number","of ratio", "pressure"]]]

CEA           Rocket Cow              delta %  \
          thermal conductivity thermal conductivity thermal conductivity   
1 chamber              0.30645             0.307308             0.280049   
  throat               0.27416             0.276473             0.843679   
  exit                 0.19803             0.201839             1.923337   
2 chamber              0.37789             0.383495             1.483248   
  throat               0.34330             0.350512             2.100885   
  exit                 0.22908             0.239034             4.345272   
3 chamber              0.39696             0.406929             2.511421   
  throat               0.37091             0.380541             2.596597   
  exit                 0.26111             0.272367             4.311155   
4 chamber              0.38919             0.398235             2.324028   
  throat               0.37064             0.378582             2.142701   
  exit                 0.28907             0.294220             1.781727   
5 chamber              0.37271             0.377736             1.348368   
  throat               0.35750             0.361180             1.029346   
  exit                 0.29831             0.296747            -0.524091   

                CEA Rocket Cow    delta %            CEA     Rocket Cow  \
          viscosity  viscosity  viscosity prandtl number prandtl number   
1 chamber  0.000057   0.000054  -5.018490         0.5076       0.480774   
  throat   0.000052   0.000049  -5.127091         0.5050       0.475096   
  exit     0.000039   0.000037  -6.115764         0.4974       0.458148   
2 chamber  0.000080   0.000075  -6.295982         0.5681       0.524550   
  throat   0.000074   0.000070  -6.221359         0.5694       0.522956   
  exit     0.000053   0.000050  -7.102992         0.5577       0.496485   
3 chamber  0.000097   0.000090  -7.405058         0.6202       0.560218   
  throat   0.000092   0.000085  -7.449335         0.6244       0.563281   
  exit     0.000070   0.000064  -7.938316         0.6311       0.557024   
4 chamber  0.000106   0.000097  -8.128264         0.6556       0.588551   
  throat   0.000102   0.000094  -8.407321         0.6613       0.592970   
  exit     0.000085   0.000077  -9.547900         0.6821       0.606214   
5 chamber  0.000110   0.000100  -8.365129         0.6745       0.609838   
  throat   0.000106   0.000097  -8.739376         0.6801       0.614302   
  exit     0.000094   0.000084 -10.721439         0.7042       0.631932   

                 delta %  
          prandtl number  
1 chamber      -5.284801  
  throat       -5.921627  
  exit         -7.891337  
2 chamber      -7.665839  
  throat       -8.156697  
  exit        -10.976404  
3 chamber      -9.671426  
  throat       -9.788421  
  exit        -11.737585  
4 chamber     -10.227078  
  throat      -10.332614  
  exit        -11.125326  
5 chamber      -9.586635  
  throat       -9.674715  
  exit        -10.262468

In [ ]:
print(CEA_composition[1]['exit'])
print(rocketcow_composition[1]['exit'])

{'CH4': 0.02205, 'CO': 0.24398, 'CO2': 0.082, 'H': 0.0, 'H2': 0.5365, 'H2O': 0.11547}
{'CH4': 0.021903470253156584, 'CO': 0.24399889108347134, 'CO2': 0.08203305957739854, 'H2': 0.5367902174493352, 'H2O': 0.115274053430428}


In [ ]:
print(CEA_composition[3]['chamber'])
print(rocketcow_composition[3]['chamber'])

{'CO': 0.26291, 'CO2': 0.06373, 'H': 0.02184, 'HCO': 1e-05, 'H2': 0.22369, 'H2O': 0.41098, 'O': 0.00084, 'OH': 0.0154, 'O2': 0.00061}
{'CO': 0.2628730935253577, 'CO2': 0.06373373212510684, 'H': 0.021929029944845496, 'H2': 0.22363090726411847, 'H2O': 0.41090583749993786, 'O': 0.0008461204028393965, 'O2': 0.0006114414465182343, 'OH': 0.015454409582575358}


In [ ]:
# def pypropep_to_dataframe(p, ox, fuel):

#     Parameters = ['of (wt ratio)', 'p (psi)', 't (K)', 'rho (kg/m^3)', 'v (m/s)', 'Isp (s)', 'Ivac (m/s)', 'c* (m/s)', 'cf', 'sound (m/s)', 'A/At', 'cp (kJ/kg-K)', 'cv (kJ/kg-K)', 'gamma', 'mol mass (g/mol)', 
#               'h (kJ/kg)', 'u (kJ/kg)', 'g (kJ/kg)', 's (kJ/kg-K)', 'dV_P', 'dV_T', 'composition']
#     positions = ['chamber', 'throat', 'exit']

#     df = pd.DataFrame(columns=Parameters, index=positions, dtype=float)

#     df.attrs = {'ox' : [ox.formula(), ox['name'], ox['id']], 
#                 'fuel' : [fuel.formula(), fuel['name'], fuel['id']]}

#     for i, c in enumerate(positions):
#         composition = p.composition[c][0:8]
#         if(bool(p.composition_condensed[c])):
#             composition.append(p.composition_condensed[c])

#         df.loc[c, 'of (wt ratio)']      = p._equil_structs[0].propellant.coef[1] * ox.mw
#         df.loc[c, 'p (atm)']            = p._equil_objs[i].properties.P
#         df.loc[c, 't (K)']              = p._equil_objs[i].properties.T
#         df.loc[c, 'rho (kg/m^3)']       = (p._equil_objs[i].properties.P * 101325 * p._equil_objs[i].properties.M / 1000) / (p._equil_objs[i].properties.T * 8.314 ) # rho (kg/m^3) = (P (atm) * 101325 (Pa) / 1 (atm) * M (g/mol) * 1 kg/1000 g)/( T (K) * R (m^3-Pa/mol-k))
#         df.loc[c, 'v (m/s)']            = p._equil_structs[i].performance.Isp
#         df.loc[c, 'Isp (s)']            = p._equil_structs[i].performance.Isp/sp.constants.g
#         df.loc[c, 'Ivac (m/s)']         = p._equil_structs[i].performance.Ivac
#         df.loc[c, 'c* (m/s)']           = p._equil_structs[i].performance.cstar
#         df.loc[c, 'cf']                 = p._equil_structs[i].performance.cf
#         df.loc[c, 'sound (m/s)']        = p._equil_structs[i].performance.ae_at
#         df.loc[c, 'A/At']               = p._equil_objs[i].properties.Vson
#         df.loc[c, 'cp (kJ/kg-K)']       = p._equil_objs[i].properties.Cp
#         df.loc[c, 'cv (kJ/kg-K)']       = p._equil_objs[i].properties.Cv
#         df.loc[c, 'gamma']              = p._equil_objs[i].properties.Isex
#         df.loc[c, 'mol mass (g/mol)']   = p._equil_objs[i].properties.M
#         df.loc[c, 'h (kJ/kg)']          = p._equil_objs[i].properties.H
#         df.loc[c, 'u (kJ/kg)']          = p._equil_objs[i].properties.U
#         df.loc[c, 'g (kJ/kg)']          = p._equil_objs[i].properties.G
#         df.loc[c, 's (kJ/kg-K)']        = p._equil_objs[i].properties.S
#         df.loc[c, 'dV_P']               = p._equil_objs[i].properties.dV_P
#         df.loc[c, 'dV_T']               = p._equil_objs[i].properties.dV_T
#         df.loc[c, 'composition']        = p.composition[c]
#         # comp_dict = {}
#         # for k, v in p.composition:
#         #     comp_dict[k] = v
#         # print(comp_dict)

#     return df

# o = ppp.PROPELLANTS['OXYGEN (LIQUID)']
# f = ppp.PROPELLANTS['METHANE']

# assumption = 'SHIFTING'
# performance = ppp.ShiftingPerformance()
# performance.add_propellants_by_mass([(f, 1.0), (o, 2)])
# performance.set_state(P=40, Pe=1)
# # df = pypropep_to_dataframe(performance, o, f)
# p = performance
# p.composition['chamber']
# for k, v in p.composition['chamber']:
#     print(k, v)

# def ranged_sim_pypropep(ox, fuel, of_arr, p_arr , p_e = 1, assumption = 'SHIFTING'):

#     # iterates through OF ratios and pressures. 
#     df_list = [] 
#     o = ppp.PROPELLANTS[ox]
#     f = ppp.PROPELLANTS[fuel]

#     if assumption == 'SHIFTING':
#         for p in p_arr:
#             for of in of_arr:
#                 # print(p, of)
#                 performance = ppp.ShiftingPerformance()
#                 performance.add_propellants_by_mass([(f, 1.0), (o, of)])
#                 performance.set_state(P=p, Pe=p_e)
#                 df = pypropep_to_dataframe(performance, o, f)
#                 df_list.append(df)
        
#     elif assumption == 'FROZEN':
#         for p in p_arr:
#             for of in of_arr:
#                 # # print(p, of)
#                 # print(f)
#                 performance = ppp.FrozenPerformance()
#                 performance.add_propellants_by_mass([(f, 1.0), (o, of)])
#                 performance.set_state(P = p, Pe = p_e)
#                 # print(performance)
#                 df = pypropep_to_dataframe(performance, o, f)
#                 df_list.append(df)    
                  
#     else: 
#         raise Exception('invalid assumption, opt ions are \'SHIFTING\' or \'FROZEN\'')
        
#     results = pd.concat(df_list, keys = list(range(len(df_list)))) 
    
#     # if not a list of dataframes is output
#     return results

# of_arr = np.arange(1, 5, 1)
# p_arr = np.arange(400/14.696, 1001/14.696, 100/14.696)
# pypropep_results = ranged_sim_pypropep('OXYGEN (LIQUID)', 'METHANE', of_arr, p_arr, assumption='FROZEN')
# pypropep_results